## How to run saved sup3r models on ERA5 data

In [79]:
import logging
import os
import xarray as xr
import hvplot.xarray
from rex import init_logger
from sup3r.models import Sup3rGanWithObs
from sup3r.preprocessing import DataHandler
import numpy as np
import matplotlib.pyplot as plt

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

init_logger('sup3r', log_level='DEBUG')
init_logger(__name__, log_level='DEBUG')
logger = logging.getLogger(__name__)

In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load a model

In [288]:
model_dir = '/datasets/sup3rwind/models/experimental/15x_1x_14f_st_obs_unet_just_embed/gan_e1300'
model = Sup3rGanWithObs.load(model_dir)

INFO - 2025-04-08 20:09:36,501 [base.py:180] : Loading GAN from disk in directory: /datasets/sup3rwind/models/experimental/15x_1x_14f_st_obs_unet_just_embed/gan_e1300
INFO - 2025-04-08 20:09:36,502 [base.py:186] : Active python environment versions: 
{   'cftime': '1.6.4',
    'dask': '2024.11.2',
    'h5netcdf': '1.3.0',
    'netCDF4': '1.6.5',
    'nrel-phygnn': '0.0.30',
    'nrel-rex': '0.2.99',
    'numpy': '1.26.4',
    'pandas': '2.2.2',
    'python': '3.11.9 (main, Apr 19 2024, 16:48:06) [GCC 11.2.0]',
    'sklearn': '1.5.1',
    'sup3r': '0.2.3.dev34+g9d6a51d2',
    'tensorflow': '2.15.1',
    'xarray': '2025.1.2'}


INFO - 2025-04-08 20:09:36,522 [abstract.py:403] : Loading model from disk that was created with the following package versions: 
{ 'cftime': '1.6.4',
  'dask': '2024.11.2',
  'h5netcdf': '1.3.0',
  'netCDF4': '1.6.5',
  'nrel-phygnn': '0.0.30',
  'nrel-rex': '0.2.99',
  'numpy': '1.26.4',
  'pandas': '2.2.2',
  'python': '3.11.9 (main, Apr 19 2024, 16:48:06) [GCC 11.2.0]',
  'sklearn': '1.5.1',
  'sup3r': '0.2.3.dev34+g9d6a51d2',
  'tensorflow': '2.15.1',
  'xarray': '2025.1.2'}


## Double check what the model needs as input

### Models take mainly low resolution data

In [289]:
model.lr_features

['temperature_2m',
 'relativehumidity_2m',
 'pressure_0m',
 'ie',
 'zust',
 'slhf',
 'sshf',
 'd2m',
 'cape',
 'kx',
 'i10fg',
 'u_10m',
 'v_10m',
 'u_100m',
 'v_100m',
 'u_200m',
 'v_200m',
 'srl',
 'sza',
 'topography']

### Models can also injest some high-resolution data (hr_exo_features).

In [290]:
model.hr_exo_features

['srl', 'sza', 'topography']

### Finaly, some models can also injest observation data which is mostly NaN except for sparse locations

In [291]:
model.obs_features

['u_10m', 'v_10m', 'temperature_2m', 'relativehumidity_2m', 'pressure_0m']

## Ok, so we need to load low-res data accordings to lr_features, and high-res data according to hr_exo_features and obs_features

#### We'll use some of the high-resolution training data as a proxy for "observation data"

In [292]:
lr_data = DataHandler('/datasets/sup3rwind/training_data/15x_to_2km_2011_*450x1200*/lr*.h5', time_slice=slice(20, 30),
                      features=model.lr_features)
hr_data = DataHandler('/datasets/sup3rwind/training_data/15x_to_2km_2011_*450x1200*/hr*.h5', time_slice=slice(20, 30),
                      features=model.hr_exo_features+[feat.replace('_obs', '') for feat in model.obs_features])

INFO - 2025-04-08 20:09:49,906 [utilities.py:125] : Initialized DataHandler with:
{ 'BaseLoader': None,
  'FeatureRegistry': None,
  'cache_kwargs': None,
  'chunks': 'auto',
  'features': [ 'temperature_2m',
                'relativehumidity_2m',
                'pressure_0m',
                'ie',
                'zust',
                'slhf',
                'sshf',
                'd2m',
                'cape',
                'kx',
                'i10fg',
                'u_10m',
                'v_10m',
                'u_100m',
                'v_100m',
                'u_200m',
                'v_200m',
                'srl',
                'sza',
                'topography'],
  'file_paths': '/datasets/sup3rwind/training_data/15x_to_2km_2011_*450x1200*/lr*.h5',
  'hr_spatial_coarsen': 1,
  'interp_kwargs': None,
  'load_features': 'all',
  'nan_method_kwargs': None,
  'res_kwargs': None,
  'shape': None,
  'target': None,
  'threshold': None,
  'time_roll': 0,
  'time_shif

/home/bbenton/repos/rex/rex/resource.py:1287: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datetime_index = pd.to_datetime(time_index.astype(str))


INFO - 2025-04-08 20:09:56,403 [base.py:153] : Getting raster index for target / shape: None / None
INFO - 2025-04-08 20:09:56,560 [base.py:224] : The distance between the closest coordinate: [  15.127 -104.881] and the requested target: [  15.127 -104.881] for files: ['/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/lr_cape.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/lr_d2m.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/lr_i10fg.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/lr_ie.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/lr_kx.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/lr_lsm.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/lr_pressure_0m.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/lr_pressure_100m.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/lr_

/home/bbenton/repos/rex/rex/resource.py:1287: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datetime_index = pd.to_datetime(time_index.astype(str))


INFO - 2025-04-08 20:10:00,012 [base.py:153] : Getting raster index for target / shape: None / None
INFO - 2025-04-08 20:10:00,069 [base.py:224] : The distance between the closest coordinate: [  14.996 -105.005] and the requested target: [  14.996 -105.005] for files: ['/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/hr_pressure_0m.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/hr_relativehumidity_2m.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/hr_srl.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/hr_sza.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/hr_temperature_2m.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/hr_topography.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/hr_u_100m.h5', '/datasets/sup3rwind/training_data/15x_to_2km_2011_15_-105_450x1200/hr_u_10m.h5', '/datasets/sup3rwind/training_data/15x_to_2

### Now we need to pass the data to the model in the format it expects

#### The 'steps' list to accomodate possible multi-step models. 'combine_type' tells the model how to incorporate the exogenous data (e.g. whether it's part of a mid-network layer or incorporated as part of a multi-step forward pass)

In [293]:
lr_input = lr_data.values[None, ...] # add batch dimension

# simulate obs data by masking some of the high-res ground truth data
onshore_mask = model._get_obs_mask(
    np.zeros((1, *hr_data.shape[:-1])), spatial_frac=1e-4, time_frac=0.8)
offshore_mask = model._get_obs_mask(
    np.zeros((1, *hr_data.shape[:-1])), spatial_frac=1e-4, time_frac=0.5)
mask = np.where(hr_data['topography'].isel(time=0).values[None] > 0,
                onshore_mask, offshore_mask)

exo_input = {}

for feature in model.hr_exo_features + model.obs_features:
    if feature in model.obs_features:
        data = np.where(
            mask[0, ..., None], np.nan,
            hr_data[feature.replace('_obs', '')].values)
    else:
        data = hr_data[feature].values
    exo_input[feature] = {
        'steps': [
            {'model': 0,
             'combine_type': 'layer',
             'data': data[None, ...][..., None],
            }
        ]
    }

### Now we can finally send the data through the model and generate output

In [294]:
hr_output = model.generate(lr_input, exogenous_data=exo_input)

### Double check ordering of output features

In [295]:
model.hr_out_features

['temperature_2m',
 'relativehumidity_2m',
 'pressure_0m',
 'u_10m',
 'v_10m',
 'u_40m',
 'v_40m',
 'u_80m',
 'v_80m',
 'u_100m',
 'v_100m',
 'u_120m',
 'v_120m',
 'u_160m',
 'v_160m',
 'u_200m',
 'v_200m']

In [296]:
dims = ('latitude', 'longitude', 'time')
coords = {
    'latitude': hr_data.latitude.values[:, 0],
    'longitude': hr_data.longitude.values[0, :],
    'time': hr_data.time
}
hr_out = xr.Dataset({feature: (dims, hr_output[0, ..., i]) for i, feature in enumerate(model.hr_out_features)})
hr_out = hr_out.assign_coords(coords)

### Let's plot windspeed at 10m and 100m

In [297]:
hr_out['windspeed_10m'] = (dims, np.hypot(hr_out['u_10m'].values, hr_out['v_10m'].values))
hr_out['windspeed_100m'] = (dims, np.hypot(hr_out['u_100m'].values, hr_out['v_100m'].values))

In [298]:
hr_out['windspeed_10m'].hvplot.quadmesh(
    groupby='time', cmap='YlGnBu', clim=(0, 10),
    rasterize=True,
    widget_type="scrubber",
    widget_location="bottom")

BokehModel(combine_events=True, render_bundle={'docs_json': {'d9a0839b-df62-4939-8602-c3a67b9b838d': {'version…

In [299]:
hr_out['windspeed_100m'].hvplot.quadmesh(
    groupby='time', cmap='YlGnBu', clim=(0, 10),
    rasterize=True,
    widget_type="scrubber",
    widget_location="bottom")

BokehModel(combine_events=True, render_bundle={'docs_json': {'dfed3acb-d406-4da4-909c-791dfb300de2': {'version…